# データ取得(NVIDIA)

# Import

In [1]:
import yfinance as yf
import pandas as pd
import sys
sys.path.append("./src")

In [2]:
import PreProcess
import Learn
import DataSets
import Models

In [ ]:
#run if needed
# NVIDIAのティッカーシンボル（NVDA）
ticker = "NVDA"
# 20年分のデータを取得
data = yf.download(ticker, start="2004-01-01", end="2024-01-01")
# データを確認
print(data.head())

In [ ]:
filename = "nvidia_20y_stock_data.csv"
data.to_csv(f"./RawData/{filename}")

In [ ]:
filename = "nvidia_20y_stock_data.csv"
data = pd.read_csv(f"./RawData/{filename}")
print(data.columns)

# PreProcess

In [ ]:
#load csv file
input_path = "./RawData/nvidia"
output_path = "./PreProcessedData/nvidia"
train_rate = 0.8
eval_rate = 0.1
test_rate = 0.1
frames_num = 1 # LSTMならseq_lenで設定する
pred_future_frame = 1 #次の日 or 3日後 (恐らく土日を挟んでいる)
target_cols =  ['Adj Close', 'Close',  'High', 'Low',  'Open', 'Volume']

PreProcess = PreProcess.PreProcess(train_rate = train_rate, eval_rate = eval_rate, test_rate = test_rate)
PreProcess.preprocess_edit(input_path = input_path, output_path = output_path, pred_future_frame = pred_future_frame, target_cols = target_cols)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.06it/s]


# Learn

In [3]:
points_num = 6
frames_num = 1
pred_future_frame = 1
batch_size = 10
modelSaveSpan = 10
folder_path = "PreProcessedData/nvidia"
learn = Learn.Learn(points_num = points_num, frames_num = frames_num, pred_future_frame = pred_future_frame, batch_size = batch_size, modelSaveSpan = modelSaveSpan)
loader = learn.data_loader_forLSTM(folder_path = folder_path, DataSetsClass = DataSets.StockPriceDataSet_forLSTM)
criterion, optimizer = learn.network_forLSTM(learning_Model = Models.StockPriceLSTM(input_size = points_num, hidden_size = 32, num_layers = 3, dropout = 0.1, seq_len = 15, output_size = 6))
learn.learn(criterion=criterion, optimizer=optimizer, n_epochs = 2500)

tensorboard --logdir=runs/Aug_CF_1_20241109171847


AttributeError: 'StockPriceDataSet_forLSTM' object has no attribute 'device'